<a href="https://colab.research.google.com/github/tengyaolong2000/SDG_Hackathon/blob/main/Demo_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the model

In [2]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 497 kB 5.0 MB/s 
     |████████████████████████████████| 428 kB 55.0 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

from tensorflow.keras.models import load_model
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.util import img_as_float
from PIL import Image

IMG_DIR = "/content/gdrive/MyDrive/Hackathon/user_photo.jpg"

Mounted at /content/gdrive


Load the model with trained parameters

In [13]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions

model = ResNet50()

In [14]:
def predict():
  #myimage = Image.open(IMG_DIR)
  myimage = load_img('/content/gdrive/MyDrive/Hackathon/user_photo.jpg', target_size=(224,224))
  myimage = img_to_array(myimage)
  myimage = myimage.reshape((1,224,224,3))
  myimage = preprocess_input(myimage)
  myresult = model.predict(myimage)
  mylabel = decode_predictions(myresult)
  mylabel = mylabel[0][0]
  return mylabel[1]



# Creating the Telegram Bot


In [15]:
from telegram.ext import Updater, CallbackContext, CommandHandler, MessageHandler, Filters
from telegram import Update

In [16]:
updater = Updater(token='5113184169:AAEljkEYTaj6PRMTZvEhqV7OLPMWsF36c3U', use_context=True)
dispatcher = updater.dispatcher

In [17]:
def start(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Hello! Please send me images of your food!")

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [18]:
# Function that is called when a photo is sent
# Latest photo is retrieved and sent to the loaded model for prediction
# Finally, the returned prediction is sent back to the user

def photo(update: Update, context: CallbackContext):
    food_dict = {'pizza': '~140 calories per slice', 'banana': '89 calories/100g', 'carbonara': '574 calories/serving'}
    context.bot.send_message(chat_id=update.effective_chat.id, text="Photo received!")
    photo_file = update.message.photo[-1].get_file()
    photo_file.download(IMG_DIR)
    output=predict()
    context.bot.send_message(chat_id=update.effective_chat.id, text="Food Prediction: " + output)
    if output.lower() in food_dict:
       context.bot.send_message(chat_id=update.effective_chat.id, text=food_dict[output])




photo_handler = MessageHandler(Filters.photo, photo)
dispatcher.add_handler(photo_handler)

In [19]:
updater.start_polling()

In [12]:
updater.stop()